<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)">Transfer Learning</h2>

In [1]:
#!pip install -q -U tensorflow
#!pip install tqdm

### Downloading the Dogs vs Cats dataset 

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

In [2]:
import requests
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
r = requests.get(url, allow_redirects=True)
open('cats_and_dogs_filtered.zip', 'wb').write(r.content)

68606236

## Dataset preprocessing

### Import project dependencies

In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [4]:
print(tf.__version__)
try:
    physical_devices = tf.config.list_physical_devices('GPU') 
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(physical_devices)
except:
    print("No GPU")

2.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-02-19 11:02:03.522440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 11:02:03.549386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 11:02:03.549650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


### Unzipping the Dogs vs Cats dataset

In [5]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

### Seting up dataset paths

In [6]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [7]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model [MobileNetV2](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2) from [Keras Applications](https://www.tensorflow.org/api_docs/python/tf/keras/applications)  

In [8]:
IMG_SHAPE = (128, 128, 3)

In [9]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

2022-02-19 11:02:04.089193: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-19 11:02:04.089774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 11:02:04.089970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 11:02:04.090081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

9420800/9406464 [==============================] - 2s 0us/step


In [10]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

### Freezing the base model

In [11]:
base_model.trainable = False

### Defining the custom head for our network

In [12]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [13]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [14]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [15]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [16]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

### Compiling the model

In [18]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/home/renato/Documents/env_default/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [19]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [20]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [21]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [22]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5


2022-02-19 11:02:10.380949: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-02-19 11:02:10.693193: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


16/16 [==============================] - 6s 210ms/step - loss: 0.6320 - accuracy: 0.6465 - val_loss: 0.5487 - val_accuracy: 0.7170
Epoch 2/5
16/16 [==============================] - 3s 160ms/step - loss: 0.5086 - accuracy: 0.7525 - val_loss: 0.4650 - val_accuracy: 0.7900
Epoch 3/5
16/16 [==============================] - 3s 158ms/step - loss: 0.4397 - accuracy: 0.8210 - val_loss: 0.4047 - val_accuracy: 0.8380
Epoch 4/5
16/16 [==============================] - 3s 168ms/step - loss: 0.3861 - accuracy: 0.8550 - val_loss: 0.3564 - val_accuracy: 0.8770
Epoch 5/5
16/16 [==============================] - 3s 162ms/step - loss: 0.3431 - accuracy: 0.8775 - val_loss: 0.3173 - val_accuracy: 0.9000


### Transfer learning model evaluation

In [23]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 1s 100ms/step - loss: 0.3173 - accuracy: 0.9000


In [24]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8999999761581421


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [25]:
base_model.trainable = True

In [26]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 154


In [27]:
fine_tune_at = 100

In [28]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [29]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [30]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 6s 203ms/step - loss: 0.2024 - accuracy: 0.9205 - val_loss: 0.0872 - val_accuracy: 0.9680
Epoch 2/5
16/16 [==============================] - 3s 158ms/step - loss: 0.0273 - accuracy: 0.9965 - val_loss: 0.0852 - val_accuracy: 0.9690
Epoch 3/5
16/16 [==============================] - 2s 154ms/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0924 - val_accuracy: 0.9710
Epoch 4/5
16/16 [==============================] - 3s 162ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0953 - val_accuracy: 0.9710
Epoch 5/5
16/16 [==============================] - 3s 158ms/step - loss: 0.0040 - accuracy: 0.9995 - val_loss: 0.1240 - val_accuracy: 0.9680


### Evaluating the fine tuned model

In [31]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/tmp/ipykernel_129384/224695654.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [32]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9679999947547913
